<a href="https://colab.research.google.com/github/carlos-alves-one/-Crypto-Electronic-Medical-Records/blob/main/crypto_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Blockchain Programming
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### Programming Assignment

## Import Cryptography Libraries

In [1]:
# Import the 'hashes' module from the cryptography library's hazmat primitives, used for cryptographic hashing
from cryptography.hazmat.primitives import hashes

# Import 'Encoding', 'PublicFormat', and 'load_der_public_key' from the serialization module for encoding formats,
# public key formats, and loading DER-encoded public keys, respectively
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, load_der_public_key

# Import the 'ec' (Elliptic Curve cryptography) and 'utils' modules from the asymmetric part of the hazmat primitives
# for performing operations with elliptic curves and various utility functions
from cryptography.hazmat.primitives.asymmetric import ec, utils


# The EMR Class

The EMR class represents an Electronic Medical Record system with functionalities to initialize records and verify their integrity and authenticity. It includes crucial operations such as hash validation, prescription validation, nonce verification, transaction ID (txid) calculation, and digital signature verification.

In [2]:
class EMR:
    def __init__(self, Dr_hash, Patient_hash, Dr_public_key, prescription, nonce, signature, txid):

        # Initialize the EMR instance with doctor's and patient's hashes, doctor's public key,
        # prescription details, nonce, digital signature, and transaction ID
        self.Dr_hash = Dr_hash
        self.Patient_hash = Patient_hash
        self.Dr_public_key = Dr_public_key
        self.prescription = prescription
        self.nonce = nonce
        self.signature = signature
        self.txid = txid

    def verify(self, Dr_previous_nonce):
        # Perform several checks to verify the integrity and authenticity of the EMR data

        # Check the length of doctor's and patient's hashes
        if len(self.Dr_hash) != 20 or len(self.Patient_hash) != 20:
            raise Exception("Hash is wrong length")

        # Validate the doctor's public key by comparing its hash against the provided doctor's hash
        if calculate_sha1_hash(self.Dr_public_key) != self.Dr_hash:
            raise Exception("Invalid doctor public key")

        # Ensure the prescription text is valid and within the allowed byte size
        if not isinstance(self.prescription, str) or len(self.prescription.encode('utf-8')) > 200:
            raise Exception("Invalid prescription")

        # Check if the nonce is sequentially correct
        if self.nonce != Dr_previous_nonce + 1:
            raise Exception("Invalid nonce")

        # Validate the transaction ID by recalculating it and comparing with the provided value
        expected_txid = calculate_txid(self.Dr_hash, self.Patient_hash, self.Dr_public_key, self.prescription, self.nonce, self.signature)
        if self.txid != expected_txid:
            raise Exception("Invalid txid")

        # Verify the digital signature to ensure the data's integrity and authenticity
        signature_hash = calculate_signature_hash(self.Patient_hash, self.prescription, self.nonce)
        key = load_der_public_key(self.Dr_public_key)
        try:
            key.verify(self.signature, signature_hash, ec.ECDSA(utils.Prehashed(hashes.SHA256())))
        except:
            raise Exception("Invalid signature")


## Supporting Functions

> This function calculate_sha1_hash computes the SHA-1 hash of the provided data. SHA-1 generates a 160-bit (20-byte) hash, which is part of various security protocols and systems.

In [3]:
def calculate_sha1_hash(data):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(data)
    return digest.finalize()


> This function calculate_txid generates a transaction ID using SHA-256 by hashing together the doctor's hash, patient's hash, doctor's public key, prescription details, nonce, and digital signature.

In [4]:
def calculate_txid(Dr_hash, Patient_hash, Dr_public_key, prescription, nonce, signature):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(Dr_hash)
    digest.update(Patient_hash)
    digest.update(Dr_public_key)
    digest.update(prescription.encode('utf-8'))
    digest.update(nonce.to_bytes(8, byteorder='little', signed=False))
    digest.update(signature)
    return digest.finalize()


> This function calculate_signature_hash produces a hash used for signature verification, combining the patient's hash, prescription, and nonce using SHA-256.

In [5]:
def calculate_signature_hash(Patient_hash, prescription, nonce):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(Patient_hash)
    digest.update(prescription.encode('utf-8'))
    digest.update(nonce.to_bytes(8, byteorder='little', signed=False))
    return digest.finalize()


These components together form a secure system for managing and verifying electronic medical records, ensuring that they are authentic, have not been tampered with, and come from a verified source. The verification process involves checking the integrity of the provided information, such as hash lengths, the correctness of the nonce, the validity of the transaction ID, and the authenticity of the signature using an elliptic curve digital signature algorithm (ECDSA) with SHA-256 as the hash function.

# The Create Signed Record Function

This function will create a signed EMR record. It takes the doctor's private key, the patient's public key hash, the prescription, and the nonce as inputs. The function calculates the txid and signature for the new record.

In [6]:
def create_signed_record(Dr_private_key, patient_hash, prescription, nonce):

    # Convert the doctor's public key to DER format
    Dr_public_key = Dr_private_key.public_key().public_bytes(encoding=Encoding.DER, format=PublicFormat.SubjectPublicKeyInfo)

    # Calculate the doctor's public key hash
    Dr_hash = calculate_sha1_hash(Dr_public_key)

    # Prepare the data for signing
    signature_hash = calculate_signature_hash(patient_hash, prescription, nonce)

    # Sign the data
    signature = Dr_private_key.sign(signature_hash, ec.ECDSA(utils.Prehashed(hashes.SHA256())))

    # Calculate the transaction ID
    txid = calculate_txid(Dr_hash, patient_hash, Dr_public_key, prescription, nonce, signature)

    # Create and return the EMR record
    return EMR(Dr_hash, patient_hash, Dr_public_key, prescription, nonce, signature, txid)


This function performs the necessary steps to create a new EMR instance with all fields properly initialized, including the cryptographic signature and transaction ID (txid), ensuring the record's integrity and non-repudiation.